In [26]:
import pandas as pd
import json
import os
import numpy as np

In [ ]:
train_df = pd.read_csv('train.jsonl')
train_df.head()

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax

# Load your DataFrame
df = pd.read_csv("../data/filtered_sentences.csv")

# Load model and tokenizer (disable fast tokenizer to avoid SentencePiece issue)
model_name = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the target entities
entities = ["Tony", "Blair", "Tony Blair"]

# Function to get sentiment for a given entity in a sentence
def get_sentiment(sentence, entity):
    # Required input format: [CLS] entity [SEP] sentence [SEP]
    formatted_input = f"[CLS] {entity} [SEP] {sentence} [SEP]"
    encoded_input = tokenizer(formatted_input, return_tensors="pt", truncation=True, padding=True).to(device)

    with torch.no_grad():
        output = model(**encoded_input)
        probs = softmax(output.logits, dim=1).cpu().numpy()[0]
        sentiment_labels = ["Negative", "Neutral", "Postitive"]
        return sentiment_labels[probs.argmax()], round(float(probs.max()), 3)

# Function to analyze all sentences for all entities
def analyze_entities(df, entities):
    sentiment_results = []

    for sentence in df["sentences"]:
        entity_sentiments = {}
        for entity in entities:
            if entity in sentence:
                sentiment, confidence = get_sentiment(sentence, entity)
                entity_sentiments[entity] = {"sentiment": sentiment, "confidence": confidence}
        sentiment_results.append(entity_sentiments)

    df["entity_sentiments"] = sentiment_results
    return df

# Run the analysis
df = analyze_entities(df, entities)

# Output the result
print(df[["sentences", "entity_sentiments"]].head())


/Users/anna_verbytska/Documents/BBC-News-Sentiment-GenAI/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


                                           sentences  \
0  When the government announced it had reached i...   
1  Palestinian leader Mahmoud Abbas has said Tony...   
2  Tony Blair does not believe abortion should be...   
3  Tony Blair called it Tony Blair's " masochism ...   
4  Tory shadow home secretary David Davis warned ...   

                                   entity_sentiments  
0  {'Tony': {'sentiment': 'neutral', 'confidence'...  
1  {'Tony': {'sentiment': 'positive', 'confidence...  
2  {'Tony': {'sentiment': 'neutral', 'confidence'...  
3  {'Tony': {'sentiment': 'negative', 'confidence...  
4  {'Tony': {'sentiment': 'negative', 'confidence...  
